In [2]:
#Langzeitmodel - Bevölkerungssynthese
import os.path
import pandas as pd
from os.path import join
import matplotlib.style

# Ergebnisordner des Langzeitmoduls
_path = r"C:\Users\devin.feng\Desktop\Git\mobitopp-rastatt\output\rastatt\demand-data"

### Analyse der Haushalte
_HH = pd.read_csv(join(_path, "household.csv"), sep=";")
# Haushalts-Attribute
household_attributes = list(_HH.columns)
df = pd.DataFrame(household_attributes)
# Anzahl Haushalte
num_of_HH = len(_HH)
# Haushalte pro Zelle
HH_per_zone = _HH.homeZone.value_counts()
# Anzahl der Pkw pro Haushalt
cars_per_household = _HH.totalNumberOfCars.value_counts()

print("HH Attributes Spalten: ", household_attributes)
print("Num. HHs: ", num_of_HH)
print("Avg. HH pro Zelle: ", HH_per_zone.mean())
print("Anzahl Zellen: ", len(HH_per_zone))
print("Anzahl der Pkw pro Haushalt: ", cars_per_household)

### Analyse der Personen
_persons = pd.read_csv(join(_path, "person.csv"), sep=";")
# Personen-Attribute
person_attributes = list(_persons.columns)
# Anzahl Personen
num_of_persons = len(_persons)
# Alter
mean_age = _persons.age.mean()
# Geschlecht
gender_distr = _persons.gender.value_counts(normalize=True)
# Beschäftigung
employment_distr = _persons.employment.value_counts()
#Anzahl Personen pro Haushalt
household_size_distr = _persons.groupby("householdId").size().value_counts()

### Auswertung Mobilitaetswerkzeuge
# OeV-Zeitkarte
ticket_distr = _persons.hasCommuterTicket.value_counts()
# Fuehrerschein
license_distr = _persons.hasLicense.value_counts()

### Verschneiden der Personen- und Haushaltstabelle
### (für manche Fragestellungen erforderlich)
_merged = pd.merge(_persons, _HH, on="householdId")
_age = _merged.age #Abkürzung für das 'age' attribut
# Verteilung der Minderjaehrigen bzw. Volljaehrigen
# in die Verkehrszellen
persons_per_zone = _merged.homeZone.value_counts()
minors_per_zone = _merged[_age < 18].homeZone.value_counts()
adults_per_zone = _merged[_age >= 18].homeZone.value_counts()

print("Person Attributes Spalten: ", person_attributes)
print("Anzahl Personen: ", num_of_persons)
print("Durchschnittsalter: ", mean_age)

_persons.head(20).to_csv(os.path.join('output/generated_csvs', 'persons_df.csv'),sep=';', index=False)
_HH.head(20).to_csv(os.path.join('output/generated_csvs', 'hh_df.csv'),sep=';', index=False)
_merged.head(20).to_csv(os.path.join('output/generated_csvs', 'merged_df.csv'),sep=';', index=False)



HH Attributes Spalten:  ['householdId', 'year', 'householdNumber', 'nominalSize', 'domCode', 'type', 'homeZone', 'homeLocation', 'homeX', 'homeY', 'numberOfMinors', 'numberOfNotSimulatedChildren', 'totalNumberOfCars', 'income', 'incomeClass', 'economicalStatus', 'canChargePrivately']
Num. HHs:  46283
Avg. HH pro Zelle:  984.7446808510638
Anzahl Zellen:  47
Anzahl der Pkw pro Haushalt:  totalNumberOfCars
1    18804
2    17069
0     6442
3     3147
4      821
Name: count, dtype: int64
Person Attributes Spalten:  ['personId', 'personNumber', 'householdId', 'age', 'employment', 'gender', 'graduation', 'income', 'hasBike', 'hasAccessToCar', 'hasPersonalCar', 'hasCommuterTicket', 'hasLicense', 'preferencesSurvey', 'preferencesSimulation', 'eMobilityAcceptance', 'chargingInfluencesDestinationChoice', 'mobilityProviderCustomership']
Anzahl Personen:  104797
Durchschnittsalter:  43.935589759248835


In [3]:
# Short term - SIMULIERTE WEGE
import pandas as pd
from os.path import join
import matplotlib.style
import matplotlib.pyplot as plt

(matplotlib.style.use('fivethirtyeight'))

_path = r"C:\Users\devin.feng\Desktop\Git\mobitopp-rastatt\output"
_short_term_path = join(_path, 'rastatt/results/simulation') # muss 'rastatt/results/simulation' sein (Handbuch falsch)
_long_term_path = join(_path, 'rastatt/demand-data')
### Einstieg (Uebersicht ueber die Daten gewinnen) ###
_trip_file = join(_short_term_path, 'demandsimulationResult.csv')
_trips = pd.read_csv(_trip_file, sep=";")
# Attribute (columns) und Kopf der Trip-Tabelle
trip_attributes = _trips.columns
trips_head = _trips.head()

# Anzahl aller Wege
num_of_trips = len(_trips)
print(trips_head)
print(trip_attributes)
# Verteilung der Wege auf die Personen
_trips_per_person = _trips.groupby('personOid').size()
_trips_per_person.hist(bins=10, rwidth=0.9)#Histogramm-Plot
plt.xlabel('Anzahl Wege in der Woche (?)')
plt.ylabel('Personen')
plt.savefig('Wege pro Woche.png', bbox_inches='tight')
plt.close()

num_of_trips_distr = _trips_per_person.value_counts()


   tripId  legId  personnumber  householdOid  personOid tripBeginDay  \
0       5      0             2         35280      81094          Mo.   
1       7      0             1          2610       5304          Mo.   
2      11      0             3          1878       4210          Mo.   
3      12      0             1          2642       5368          Mo.   
4      20      0             3         32312      73229          Mo.   

   activityNumber tripBeginTime  activityType  previousActivityType  ...  \
0               2      00:01:00            52                     7  ...   
1               2      00:02:00            12                     7  ...   
2               2      00:05:00             1                     7  ...   
3               2      00:05:00            11                     7  ...   
4               2      00:05:00            42                     7  ...   

   isMainActivity  realEndDay  realEndTime     fromX      fromY       toX  \
0               0         Mo.    

In [ ]:
### Wegebezogener Modal Split ####

# Einfache Berechnung
modal_split_abs =_trips.tripMode.value_counts()
modal_split_rel =_trips.tripMode.value_counts(normalize=True)
# Funktion zur Berechnung des Modal Split erstellen
# Zweck: folgenden Code uebersichtlicher halten
def modal_split(trip_df):
    return trip_df.tripMode.value_counts(normalize=True)
test_modal_split_funtion = modal_split(_trips)
# Modal Split fuer Wege innerhalb Rastatt
# (Start und Ziel in Zellen mit Nr. kleiner 6200)
_sourceZone = _trips.sourceZone
_targetZone = _trips.targetZone
modal_split_in_rastatt = modal_split(_trips[(_sourceZone < 6200)
& (_targetZone < 6200)])
# Modal Split auf Arbeitswegen
_activityType = _trips.activityType
modal_split_work = modal_split(_trips[_activityType == 1])
